In [1]:
import numpy as np
import cv2 as cv
import os
from datetime import datetime
from matplotlib import pyplot as plt
from easyocr import Reader

In [2]:
people = ['car']

# direction to the folder with images
DIR = r'E:\images'

haar = cv.CascadeClassifier('haar_plate.xml')

features = []
labels = []

def create_train():
    for person in people:
        path = os.path.join(DIR, person)
        label = people.index(person)

        for img in os.listdir(path):
            img_path = os.path.join(path, img)

            img_array = cv.imread(img_path)
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)

            faces_rect = haar.detectMultiScale(gray)

            for (x, y, w, h) in faces_rect:
                faces_roi = gray[y:y+h, x:x+w]
                features.append(faces_roi)
                labels.append(label)

In [3]:
create_train()

print('НАТРЕНИРОВАНО! ---------------')

features = np.array(features, dtype='object')
labels = np.array(labels)

face_recognizer = cv.face.LBPHFaceRecognizer_create()

# Train the Recognizer on the features list and the labels list
face_recognizer.train(features,labels)

face_recognizer.save('plate_trained.yml')
np.save('features.npy', features)
np.save('labels.npy', labels)

НАТРЕНИРОВАНО! ---------------


In [27]:
haar_cascade = cv.CascadeClassifier('haar_plate.xml')

people = ['car']
#features = np.load('features.npy', allow_pickle=True)
# labels = np.load('labels.npy')

face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read('face_trained.yml')

img = cv.imread(r'E:\images\car\1.jpg')

gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)



# Detect the face in the image
faces_rect = haar_cascade.detectMultiScale(gray)

for (x,y,w,h) in faces_rect:
    faces_roi = gray[y:y+h,x:x+w]
    reader = Reader(['en'])
    license_plate = gray[y:y + h, x:x + w]
    detection = reader.readtext(license_plate)
    
    contours, _ = cv.findContours(edged, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv.contourArea, reverse = True)[:5]


    for c in contours:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            n_plate_cnt = approx
            break        

    (x, y, w, h) = cv.boundingRect(n_plate_cnt)
    
    
    
    label, confidence = face_recognizer.predict(faces_roi)
    print(f'Label = {people[label]} with a confidence of {confidence}')

    if len(detection) == 0:
        text = "Impossible to find the cars license plate"
        cv.putText(img, text, (20, 40), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)
    #     cv2.imshow('Image', image)
    #     cv2.waitKey(0)
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

    else:
        cv.drawContours(img, [n_plate_cnt], -1, (0, 255, 0), 3)
        text = f"{detection[0][1]} {detection[0][2] * 100:.2f}%"
        cv.putText(img, text, (x, y - 20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    
    cv.putText(img, str(people[label]), (20,20), cv.FONT_HERSHEY_COMPLEX, 1.0, (0,255,0), thickness=2)
    cv.rectangle(img, (x,y), (x+w,y+h), (0,255,0), thickness=2)

cv.imshow('Detected Face', img)

cv.waitKey(0)

cv.destroyAllWindows()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


NameError: name 'edged' is not defined

In [4]:
import numpy as np

face = cv.CascadeClassifier('haar_plate.xml')
people = ['car']
vid = cv.VideoCapture(0)
face_recognizer = cv.face.LBPHFaceRecognizer_create()
face_recognizer.read('plate_trained.yml')

while 1:
    
    ret, img = vid.read()
    img = cv.flip(img, 1)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray, 1.3, 5)
    

    for (x,y,w,h) in faces:
        license_plate = gray[y:y+h,x:x+w]
        label, confidence = face_recognizer.predict(license_plate)
        print(f'Label = {people[label]} with a confidence of {confidence}')
        
        reader = Reader(['en'])
        detection = reader.readtext(license_plate)

        if len(detection) == 0:
            text = "Impossible to find the cars license plate"
            cv.putText(img, text, (20, 40), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 3)
        #     cv2.imshow('Image', image)
        #     cv2.waitKey(0)
            plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

        else:
            cv.drawContours(img, [n_plate_cnt], -1, (0, 255, 0), 3)
            text = f"{detection[0][1]} {detection[0][2] * 100:.2f}%"
            cv.putText(img, text, (x, y - 20), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
        
        cv.putText(img, str(people[label]), (20,20), cv.FONT_HERSHEY_COMPLEX, 1.0, (255,0,0), thickness=2)
        cv.rectangle(img,(x,y),(x+w,y+h),(0,204,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        print (int(x+w/2), int(y+h/2))
        
    cv.imshow('img',img)
    k = cv.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break

vid.release()
cv.destroyAllWindows()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label = car with a confidence of 107.72079839508274


error: Unknown C++ exception from OpenCV code